In [ ]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define constants
train_dir = 'tomato/train/'
test_dir = 'tomato/val/'
batch_size = 32
num_classes = 10
input_shape = (150, 150, 3)  # Adjust according to your image dimensions

# Function to filter images based on extensions
def filter_images_by_extension(directory):
    valid_extensions = ['.jpg', '.jpeg', '.png']
    image_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if any(file.lower().endswith(ext) for ext in valid_extensions):
                image_files.append(os.path.join(root, file))
    return image_files

# Filter image files
train_image_files = filter_images_by_extension(train_dir)
test_image_files = filter_images_by_extension(test_dir)

# Create DataFrame for training and testing
train_df = pd.DataFrame({
    'filename': train_image_files,
    'class': [os.path.basename(os.path.dirname(path)) for path in train_image_files]
})

test_df = pd.DataFrame({
    'filename': test_image_files,
    'class': [os.path.basename(os.path.dirname(path)) for path in test_image_files]
})

# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Normalization for testing
test_datagen = ImageDataGenerator(rescale=1./255)

# Generating batches of training and testing data
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='filename',
    y_col='class',
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='filename',
    y_col='class',
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical')

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=10, validation_data=test_generator)



Found 10000 validated image filenames belonging to 10 classes.
Found 1000 validated image filenames belonging to 10 classes.
Epoch 1/10
313/313 [==============================] - 77s 246ms/step - loss: 1.2588 - accuracy: 0.5516 - val_loss: 1.1172 - val_accuracy: 0.6630
Epoch 2/10
313/313 [==============================] - 93s 296ms/step - loss: 0.5994 - accuracy: 0.7881 - val_loss: 0.9235 - val_accuracy: 0.7120
Epoch 3/10
313/313 [==============================] - 96s 305ms/step - loss: 0.4458 - accuracy: 0.8415 - val_loss: 0.5347 - val_accuracy: 0.8250
Epoch 4/10
313/313 [==============================] - 96s 308ms/step - loss: 0.3512 - accuracy: 0.8750 - val_loss: 0.6124 - val_accuracy: 0.8110
Epoch 5/10
313/313 [==============================] - 100s 320ms/step - loss: 0.3080 - accuracy: 0.8908 - val_loss: 0.5106 - val_accuracy: 0.8380
Epoch 6/10
313/313 [==============================] - 100s 320ms/step - loss: 0.2686 - accuracy: 0.9063 - val_loss: 0.5110 - val_accuracy: 0.8560
Epo

In [ ]:

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)

32/32 [==============================] - 2s 60ms/step - loss: 0.7217 - accuracy: 0.8080
Test accuracy: 0.8080000281333923


In [ ]:
import matplotlib.pyplot as plt
import cv2

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the image you want to predict
img_path = 'leafmold.JPG'  # Provide the path to your image
img = image.load_img(img_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array /= 255.  # Rescale pixel values to [0, 1]

# Make prediction
prediction = model.predict(img_array)
predicted_class = np.argmax(prediction)

# Map predicted class index to class label
class_labels = train_generator.class_indices
predicted_label = [k for k, v in class_labels.items() if v == predicted_class][0]

print("Predicted class:", predicted_label)
for i in range(0,10):
    prediction[0][i] = np.round((prediction[0][i]),2)
print("Prediction probabilities:", prediction)


1/1 [==============================] - 0s 19ms/step
Predicted class: Tomato___Leaf_Mold
Prediction probabilities: [[0.   0.   0.   0.99 0.   0.   0.   0.   0.01 0.  ]]


In [ ]:

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the converted model to a file
with open('disease.tflite', 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: /var/folders/wc/4vhbxzyn749cv5d6p5s6nw100000gp/T/tmp1o0dqosz/assets


INFO:tensorflow:Assets written to: /var/folders/wc/4vhbxzyn749cv5d6p5s6nw100000gp/T/tmp1o0dqosz/assets
2024-04-04 15:42:58.941670: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-04-04 15:42:58.941681: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-04-04 15:42:58.941885: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/wc/4vhbxzyn749cv5d6p5s6nw100000gp/T/tmp1o0dqosz
2024-04-04 15:42:58.942604: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-04-04 15:42:58.942608: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/wc/4vhbxzyn749cv5d6p5s6nw100000gp/T/tmp1o0dqosz
2024-04-04 15:42:58.943922: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-04-04 15:42:58.944493: I tensorflow/cc/saved_model/load